element = driver.find_element_by_link_text("Interview Preparation")

encontrar elementos por texto

XPATH -> //tag[@arg = "value"]/TagFilhaSubjacente

'//tag[contains(@attr, "nome_aproximado")]'



https://fbref.com/pt/comps/Big5/Maiores-5-Ligas-Europeias-Estatisticas

https://stackoverflow.com/questions/47316810/unable-to-locate-elements-on-webpage-with-headless-chrome

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
#import pickle
import json

In [6]:
PATH = "./driver/chromedriver"
service = Service(executable_path=PATH) # selenium 4
options = Options()
options.headless = False
#options.add_argument("window-size=1920x1980") executar com headless = True e argumento descomentado
#faz com que para algumas equipes o URL recuperado seja o do AD do google
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
options.add_argument(f'user-agent={user_agent}')

In [7]:
url = "https://fbref.com/pt/comps/Big5/Maiores-5-Ligas-Europeias-Estatisticas"
table_id = "big5_table"

In [12]:
def crawler(url,table_id):
    
    start_time = time.time()
    
    equipes = []
    urls = []

    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)

    driver.maximize_window()
    
    table = driver.find_element(By.ID, table_id)
    
    #. se refere ao caminho de table
    teams = table.find_elements(By.XPATH, './/td[@data-stat="squad"]/a') 
    #fora do loop para poder ter len(temas)

    for team in teams:
        equipes.append(team.text)

    for i in range(len(teams)): 

        table = driver.find_element(By.ID, "big5_table")

        teams = table.find_elements(By.XPATH, './/td[@data-stat="squad"]/a')

        actions = ActionChains(driver)

        #ignorar erro boundary gerado por move elemente()
        try:
            actions.move_to_element(teams[i+1]).perform()
        except:
            pass

        time.sleep(4) #espera para nao mover e clicar ao mesmo tempo

        teams[i].click()

        # clicar no AD do google -> necessita mudar para o frame do botão de fechar
        try:
            wait=WebDriverWait(driver, 4)
            wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'aswift_2')))
            ad_button = driver.find_element(By.XPATH, '//div[contains(@id, "dismiss-button")]')
            ad_button.click()
            driver.switch_to.parent_frame()
        except:
            pass

        #time.sleep(3)
        page = driver.current_url
        print(f"{equipes[i]} ----------- {page}")

        urls.append(page)

        #retornar a tabela com links de equipes para clicar na proxima 
        driver.back()

    driver.quit()

    urls_dic = {}
    for equipe, url in zip(equipes, urls):
        urls_dic[equipe] = url
    
    print(f"Tempo de execução: {round(time.time() - start_time, 2)} segundos")
    
    return urls_dic


In [4]:
urls_dic_test = crawler(url, table_id)

Manchester City ----------- https://fbref.com/pt/equipes/b8fd03ef/Manchester-City-Estatisticas
Liverpool ----------- https://fbref.com/pt/equipes/822bd0ba/Liverpool-Estatisticas
Bayern Munich ----------- https://fbref.com/pt/equipes/054efa67/Bayern-Munich-Estatisticas
Paris S-G ----------- https://fbref.com/pt/equipes/e2d8892c/Paris-Saint-Germain-Estatisticas
Real Madrid ----------- https://fbref.com/pt/equipes/53a2f082/Real-Madrid-Estatisticas
Milan ----------- https://fbref.com/pt/equipes/dc56fe14/Milan-Estatisticas
Inter ----------- https://fbref.com/pt/equipes/d609edc0/Internazionale-Estatisticas
Napoli ----------- https://fbref.com/pt/equipes/d48ad4ff/Napoli-Estatisticas
Dortmund ----------- https://fbref.com/pt/equipes/add600ae/Dortmund-Estatisticas
Chelsea ----------- https://fbref.com/pt/equipes/cff3d9bb/Chelsea-Estatisticas
Barcelona ----------- https://fbref.com/pt/equipes/206d90db/Barcelona-Estatisticas
Leverkusen ----------- https://fbref.com/pt/equipes/c7a9f859/Bayer-Lever

In [5]:
#Salvar arquivo em formato json
#a_file = open("./data/TEST_urls_dic.json", "w")
#json.dump(./data/urls_dic, a_file)
#a_file.close()

#Ler arquivo JSON
#with open('./data/TEST_urls_dic.json', 'r') as f:
#    data = json.load(f)

In [10]:
type(data)

dict

In [4]:
data["Milan"]

'https://fbref.com/pt/equipes/dc56fe14/Milan-Estatisticas'